In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd

In [ ]:
auth.authenticate_user()

In [ ]:
client = bigquery.Client(project = "continual-apex-402217")
url = "gs://tech_challenge_3/datasus_covid_2020_maio_julho.csv"
dataset_id = 'covid'
table_id = 'datasus_covid'

In [ ]:
df_raw = pd.read_csv(url, thousands='.')

In [ ]:
df_raw.head()

,Info,UF,Município,Metro/Interior,Ano_Semana,CasosAcumulados,CasosNovos,ObitosAcumulados,ObitosNovos
0,COVID19Casos,AC,Acrelândia,Interior,18/2020,90,1,0,0
1,COVID19Casos,AC,Acrelândia,Interior,19/2020,120,15,5,1
2,COVID19Casos,AC,Acrelândia,Interior,20/2020,303,39,7,0
3,COVID19Casos,AC,Acrelândia,Interior,21/2020,652,62,7,0
4,COVID19Casos,AC,Acrelândia,Interior,22/2020,1001,29,7,0


In [ ]:
# Carregue o arquivo CSV em um DataFrame
df_covid = df_raw

# Mapeie os valores de "Ano_Semana" para os valores correspondentes em "ano_mes"
mes_semana_mapping = {
    '19/2020': '5/1', '20/2020': '5/2', '21/2020': '5/3', '22/2020': '5/4',
    '23/2020': '6/1', '24/2020': '6/2', '25/2020': '6/3', '26/2020': '6/4',
    '27/2020': '7/1', '28/2020': '7/2', '29/2020': '7/3', '30/2020': '7/4', '31/2020': '7/4'
}

# Adicione a coluna "ano_mes" com base no mapeamento
df_covid['mes_semana'] = df_covid['Ano_Semana'].map(mes_semana_mapping)

# Selecione as colunas desejadas
df_covid = df_covid[['mes_semana', 'UF', 'CasosNovos', 'ObitosNovos']]

df_covid.rename(columns = {'UF':'uf', 'CasosNovos': 'casos', 'ObitosNovos': 'obitos'}, inplace=True)

# Agrupe os dados pela coluna "UF" e some os valores de "CasosNovos" e "ObitosNovos"
df_covid = df_covid.groupby(['uf', 'mes_semana']).agg({'casos': 'sum', 'obitos': 'sum'}).reset_index()

# O resultado estará no DataFrame "result"
print(df_covid)

     uf mes_semana  casos  obitos
0    AC        5/1    782      17
1    AC        5/2    532      20
2    AC        5/3   2000      28
3    AC        5/4   2205      55
4    AC        6/1   1453      59
..   ..        ...    ...     ...
319  TO        6/4   1763      24
320  TO        7/1   2316      24
321  TO        7/2   2657      36
322  TO        7/3   2270      37
323  TO        7/4   8137     102

[324 rows x 4 columns]


<ipython-input-33-9b94dc0941a3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid.rename(columns = {'UF':'uf', 'CasosNovos': 'casos', 'ObitosNovos': 'obitos'}, inplace=True)


In [ ]:
# Escreve o dataframe na tabela do BigQuery
table_ref = client.dataset(dataset_id).table(table_id)
job = client.load_table_from_dataframe(df_covid, table_ref)
job.result()

LoadJob<project=continual-apex-402217, location=US, id=bc1689ca-a691-498e-babb-7a2b0b304278>